In [1]:
import os
import pickle
import pandas as pd

In [2]:
CSV_FIELD = {
    "cpu.csv":["%user", "%nice", "%system", "%iowait", "%idle", "proc/s", "cswch/s"],
    "memory.csv": [
        "pgfree/s", "pgscand/s", "pgsteal/s", "%vmeff", "kbmemfree", "kbavail", "kbmemused",
        "%memused", "kbbuffers", "kbcached", "kbcommit", "%commit", "kbactive", "kbinact", "kbdirty"
    ],
    "disk.csv": ["tps", "rkB/s", "wkB/s", "areq-sz", "aqu-sz", "await", "%util"],
    "network.csv": [
        "total/s", "retrans/s", "totsck", "tcpsck", "udpsck", "tcp-tw", "rxpck/s", "txpck/s",
        "rxkB/s", "txkB/s", "ierr/s", "oerr/s", "active/s", "iseg/s", "oseg/s"
    ],
}


def log(*args, **kwargs):
    import datetime
    kwargs.setdefault("flush", True)
    print(datetime.datetime.now(), *args, **kwargs)

In [3]:
class Train:
    def __init__(self, dataset):
        self.dataset = dataset
        self.workload = None
        self.train_df = None

    def load(self):
        workloads = []
        train_dfs = []

        for path in os.listdir(self.dataset):
            instance_ip = path
            log("start load train", instance_ip, flush=True)
            path = os.path.join(self.dataset, path)
            assert os.path.isdir(path)
            workload_file = os.path.join(path, "workload.pkl")
            with open(workload_file, "rb") as f:
                workload = pickle.load(f)
            workloads.append(workload)
            for item in os.listdir(path):
                if item == 'workload.pkl':
                    continue
                day_path = os.path.join(path, item)
                ip_df = pd.DataFrame({"event_time": []})
                for file in os.listdir(day_path):
                    assert file.endswith(".csv")
                    df = pd.read_csv(os.path.join(day_path, file))
                    df.set_index("event_time", inplace=True)
                    ip_df = ip_df.merge(df, on="event_time", how='outer', suffixes=("", "_y"), copy=False)
                ip_df['instance_ip'] = instance_ip
                train_dfs.append(ip_df)
        self.workload = pd.concat(workloads)
        self.train_df = pd.concat(train_dfs)
        self.train_df.event_time = pd.to_datetime(self.train_df.event_time)


class Test:

    def __init__(self, dataset, train):
        self.dataset = dataset
        self.train = train
        self.workload = None
        self.df = train.train_df
        self.res_df = None

    def load(self):
        instance_ips = []
        workloads = []
        dfs = []
        for path in os.listdir(self.dataset):
            instance_ip = path
            log("start load test", instance_ip, flush=True)
            path = os.path.join(self.dataset, path)
            assert os.path.isdir(path)
            workload_file = os.path.join(path, "workload.pkl")
            with open(workload_file, "rb") as f:
                workload = pickle.load(f)
            workloads.append(workload)
            submit_range_file = os.path.join(path, "submit_range.csv")
            dfs.append(pd.read_csv(submit_range_file))
            instance_ips.append(instance_ip)
        self.workload = pd.concat(workloads)
        self.res_df = pd.concat(dfs)
        self.res_df.event_time = pd.to_datetime(self.res_df.event_time)
        min_time = min(self.df.event_time.min(), self.res_df.event_time.min())
        max_time = max(self.df.event_time.max(), self.res_df.event_time.max())
        all_df = pd.DataFrame({
            'event_time': [
                min_time + pd.Timedelta(s, "seconds")
                for s in range(int((max_time - min_time).total_seconds()) + 1)
            ]
        })
        for instance_ip in instance_ips:
            all_df['instance_ip'] = instance_ip
            self.df = self.df.merge(all_df, how='outer', on=["instance_ip", "event_time"], copy=False)

    def predict(self):
        self.df["ierr/s"] = 0
        self.df["oerr/s"] = 0
        
        # todo 预测
        self.df.fillna(0, inplace=True)

        self.res_df = self.res_df.merge(self.df, how='inner', on=["instance_ip", "event_time"], copy=False)

    def write_to_file(self, out_path='./'):
        for file, fields in CSV_FIELD.items():
            self.res_df[["instance_ip", "event_time", *fields]].to_csv(
                os.path.join(out_path, file), header=True, index=False
            )   


In [4]:
# train_a = Train("round_a_train_dataset")
# train_a.load()

In [5]:
class Train_b:
    def __init__(self, dataset_workload, dataset_indicators):
        self.dataset_workload = dataset_workload
        self.dataset_indicators = dataset_indicators
        self.workload = None
        self.train_df = None
        self.ips = []

    def load(self):
        workloads = []
        for path in os.listdir(self.dataset_workload):
            instance_ip = path
            self.ips.append(instance_ip)
            log("start load train", instance_ip, flush=True)
            path = os.path.join(self.dataset_workload, path)
            assert os.path.isdir(path)
            workload_file = os.path.join(path, "workload.pkl")
            with open(workload_file, "rb") as f:
                workload = pickle.load(f)
            workloads.append(workload)
        self.workload = pd.concat(workloads)
        self.workload.start_time = pd.to_datetime(self.workload.start_time, unit='s')
        self.workload.end_time = pd.to_datetime(self.workload.end_time, unit='s')
        time_ip_df = pd.DataFrame({"event_time": [], "instance_ip": []})
        for item in os.listdir(self.dataset_indicators):
            csv_path = os.path.join(self.dataset_indicators, item)
            assert item.endswith(".csv")
            df = pd.read_csv(csv_path)
            df.set_index(["event_time", "instance_ip"], inplace=True)
            time_ip_df = time_ip_df.merge(df, on=["event_time","instance_ip"], how="outer", suffixes=("", "_y"), copy=False)
        self.train_df = time_ip_df
        self.train_df.event_time = pd.to_datetime(self.train_df.event_time, unit='s')
    
    def dropNan(self):
        self.train_df = self.train_df.dropna(axis=0, how="any")
    
    def dfByIP(self, instance_ip):
        return self.train_df[self.train_df['instance_ip']==instance_ip].sort_values(by='event_time')
    
    def workloadByIp(self, instance_ip):
        return self.workload[self.workload['host_ip']==instance_ip].sort_values(by='start_time')


class Test_b:

    def __init__(self, dataset, train):
        self.dataset = dataset
        self.train = train
        self.workload = None
        self.df = None
        self.res_df = None
    
    def load(self):
        df = pd.DataFrame({"event_time": [], "instance_ip": []})
        for path in os.listdir(self.dataset):
            assert path.endswith(".csv")
            indicator_name = path.replace(".csv", "")
            log("start load test ", path, flush=True)
            indicator = pd.read_csv(os.path.join(self.dataset,path))
            indicator[indicator_name] = indicator_name
            df = df.merge(indicator, how='outer', on=["instance_ip", "event_time"], copy=False)
        self.df = df
        self.df.event_time = pd.to_datetime(self.df.event_time)


In [14]:
# train_b = Train_b("round_b_workload", "round_b_train_v20240715")
# train_b.load()
# with open('train_b.pickle', 'wb') as f:
#     pickle.dump(obj=train_b, file=f, protocol=4)
with open('train_b.pickle', 'rb') as f:
    train_b = pickle.load(f)
# train_b.train_df.fillna(0, inplace=True)

test_b = Test_b("round_b_test_v20240715", None)
test_b.load()

2024-07-28 05:43:22.348936 start load test  cpu.csv
2024-07-28 05:43:22.609911 start load test  disk.csv
2024-07-28 05:43:23.176379 start load test  memory.csv
2024-07-28 05:43:23.909149 start load test  network.csv


In [7]:
# for ip in train_b.ips:
#     df_t = train_b.dfByIP(ip)
#     delta = df_t['event_time'].shift(-1) - df_t['event_time']
#     print(ip, delta.max(), delta.min())

# 显示train_df中，同一台机器的状态日志的时间差最长和最短是多少
# 172.24.96.146 0 days 00:00:05 0 days 00:00:00
# 172.24.96.147 0 days 00:00:31 0 days 00:00:00
# 172.24.96.148 0 days 00:00:17 0 days 00:00:00
# 172.24.96.149 0 days 22:59:23 0 days 00:00:00
# 172.24.96.150 0 days 00:00:07 0 days 00:00:00
# 172.24.96.151 0 days 00:00:04 0 days 00:00:00
# 172.24.96.152 0 days 00:00:07 0 days 00:00:00
# 172.24.96.153 0 days 00:00:55 0 days 00:00:00

In [9]:
from enum import Enum
JobType = Enum('JobType', ('NetInject_Workload', 'NetOutputIO_Workload', 'IO_Workload', 'Memory_Workload', 'CPU_Workload',\
                           'NetIO_Workload', 'NetInputIO_Workload'), start=0)

In [10]:
# pattern = "stress-ng --cpu"
# mask = [True if i.startswith(pattern) else False for i in workload.scripts]
# cpuwl = workload[mask]['scripts']
# l = list(cpuwl)
# newl = []
# for s in l:
#     i = s.find('cpu-method ') + len('cpu-method ')
#     method = s[i:].split(' ')[0]
#     if method not in newl:
#         newl.append(method)
# print(newl)

['intconversion', 'all', 'fnv1a', 'idct', 'int16', 'rand48', 'floatconversion', 'callfunc', 'int128decimal32', 'prime', 'djb2a', 'rand', 'sdbm', 'float128', 'longdouble', 'phi', 'apery', 'queens', 'parity', 'rgb', 'explog', 'float32', 'int128', 'correlate', 'fibonacci', 'pi', 'int128longdouble', 'gray', 'crc16', 'bitops', 'hyperbolic', 'ln2', 'float80', 'zeta', 'sqrt', 'cdouble', 'loop', 'int32', 'gamma', 'jmp', 'trig', 'double', 'hanoi', 'nsqrt', 'stats', 'int32longdouble', 'int64longdouble', 'int64float', 'omega', 'div64', 'clongdouble', 'int128decimal64', 'cpuid', 'collatz', 'int128float', 'int64double', 'hamming', 'euler', 'decimal32', 'int128double', 'decimal128', 'ackermann', 'jenkin', 'float', 'int32float', 'int64', 'union', 'factorial', 'int8', 'matrixprod', 'int128decimal128']


In [11]:
# pattern = "stress-ng --vm"
# mask = [True if i.startswith(pattern) else False for i in workload.scripts]
# cpuwl = workload[mask]['scripts']
# l = list(cpuwl)
# newl = []
# for s in l:
#     i = s.find('vm-method ') + len('vm-method ')
#     method = s[i:].split(' ')[0]
#     if method not in newl:
#         newl.append(method)
# print(newl)

['modulo-x', 'walk-0a', 'zero-one', 'swap', 'galpat-0', 'move-inv', 'prime-0', 'write64', 'flip', 'ror', 'prime-gray-0', 'read64', 'walk-1d', 'rand-set', 'prime-gray-1', 'all', 'prime-1', 'rowhammer', 'inc-nybble', 'walk-0d', 'prime-incdec', 'gray', 'rand-sum', 'incdec', 'walk-1a', 'galpat-1']


In [12]:
from enum import Enum
scriptsType = Enum('scriptsType', ('netem', 'tbf', 'oss', 'hdd', 'cpu', 'vm','tmp'), start=0)
netemOffset = Enum('netOffset', ('loss', 'duplicate', 'delay', 'reorder1', 'reorder2'), start=0)
tbfOffset = Enum('tbfOffset', ('rate', 'burst', 'latency'), start=0)
ossOffset = Enum('ossOffset', ('oss'), start=0)
hddOffset = Enum('hddOffset', ('hdd', 'bytes', 'size', 'timeout'), start=0)
cpuOffset = Enum('cpuOffset', ('cpu','timeout', 'intconversion', 'all', 'fnv1a', 'idct', 'int16', 'rand48', 'floatconversion', 'callfunc', 'int128decimal32', 'prime', 'djb2a', 'rand', 'sdbm', 'float128', 'longdouble', 'phi', 'apery', 'queens', 'parity', 'rgb', 'explog', 'float32', 'int128', 'correlate', 'fibonacci', 'pi', 'int128longdouble', 'gray', 'crc16', 'bitops', 'hyperbolic', 'ln2', 'float80', 'zeta', 'sqrt', 'cdouble', 'loop', 'int32', 'gamma', 'jmp', 'trig', 'double', 'hanoi', 'nsqrt', 'stats', 'int32longdouble', 'int64longdouble', 'int64float', 'omega', 'div64', 'clongdouble', 'int128decimal64', 'cpuid', 'collatz', 'int128float', 'int64double', 'hamming', 'euler', 'decimal32', 'int128double', 'decimal128', 'ackermann', 'jenkin', 'float', 'int32float', 'int64', 'union', 'factorial', 'int8', 'matrixprod', 'int128decimal128'), start=0)
vmOffset = Enum('vmOffset', ('vm', 'bytes', 'timeout', 'modulo-x', 'walk-0a', 'zero-one', 'swap', 'galpat-0', 'move-inv', 'prime-0', 'write64', 'flip', 'ror', 'prime-gray-0', 'read64', 'walk-1d', 'rand-set', 'prime-gray-1', 'all', 'prime-1', 'rowhammer', 'inc-nybble', 'walk-0d', 'prime-incdec', 'gray', 'rand-sum', 'incdec', 'walk-1a', 'galpat-1'), start=0)
tmpOffset = Enum('tmpOffset', ('writew', 'writer', 'removew', 'remover', 'tmpw', 'tmpr'), start=0)


def makeEmptyScriptsDict():
    result = {}
    result['netem'] = {
        "ScriptsType": scriptsType.netem,
        "parameters": [i for i in range(len(netemOffset)+1)]
    }
    result['tbf'] = {
        "ScriptsType": scriptsType.tbf,
        "parameters": [i for i in range(len(tbfOffset))]
    }
    result['oss'] = {
        "ScriptsType": scriptsType.oss,
        "parameters": [i for i in range(len(ossOffset))]
    }
    result['hdd'] = {
        "ScriptsType": scriptsType.hdd,
        "parameters": [i for i in range(len(hddOffset))]
    }
    result['cpu'] = {
        "ScriptsType": scriptsType.cpu,
        "parameters": [i for i in range(len(cpuOffset))]
    }
    result['vm'] = {
        "ScriptsType": scriptsType.vm,
        "parameters": [i for i in range(len(vmOffset))]
    }
    result['tmp'] = {
        "ScriptsType": scriptsType.tmp,
        "parameters": [i for i in range(len(tmpOffset))]
    }
    return result

class ScriptsParser:
    def __init__(self):
        self.result = None
        self.scripts = None
    
    def parse(self, scripts):
        self.scripts = scripts
        if self.parse_tc():
            return self.result
        elif self.parse_oss():
            return self.result
        elif self.parse_hdd():
            return self.result
        elif self.parse_cpu():
            return self.result
        elif self.parse_vm():
            return self.result
        elif self.parse_tmp():
            return self.result
        return False
    
    
    def parser_tc_tbf(self, scripts):
        # 'tbf rate 512kbit burst 32kbit latency 3378ms'
        # scripts startswith 'tbf'
        def bitToFloat(n):
            n = n.replace("bit", "")
            base = 0
            if n.endswith("m"):
                base = 10
            elif n.endswith("k"):
                base = 1
            elif n[-1] in "abcdefghijklmnopqrstuvwxyz":
                raise ValueError("unknon tbf parameters: ", n)
            n = n[:-1]
            return float(n) * base
        scripts = scripts.split(' ')
        rate = bitToFloat(scripts[2])
        burst = bitToFloat(scripts[4])
        latency = float(scripts[6].replace("ms", ""))
        parameters = [0 for i in range(len(tbfOffset))]
        parameters[tbfOffset.rate.value] = rate
        parameters[tbfOffset.burst.value] = burst
        parameters[tbfOffset.latency.value] = latency
        self.result = {
            "ScriptsType": scriptsType.tbf,
            "parameters": parameters
        }
    
    def parse_tc_netem(self, scripts):
        # scripts startswith 'netem'
        scripts = scripts.split(' ')[1:]
        idx = 0
        parameters = [0 for i in range(len(netemOffset)+1)]
        while idx < len(scripts):
            if scripts[idx] == 'loss':
                loss = float(scripts[idx + 1].replace("%", ""))
                parameters[netemOffset.loss.value] = loss
                idx += 2
            elif scripts[idx] == 'delay':
                delay = float(scripts[idx+1].replace("ms", ""))
                parameters[netemOffset.delay.value] = delay
                idx += 2
            elif scripts[idx] == 'duplicate':
                duplicate = float(scripts[idx + 1].replace("%", ""))
                parameters[netemOffset.duplicate.value] = duplicate
                idx += 2
            elif scripts[idx] == 'reorder':
                ratio = float(scripts[idx+1].replace("%", ""))
                parameters[netemOffset.reorder1.value] = ratio
                ms = float(scripts[idx+2].replace("ms", ""))
                parameters[netemOffset.reorder2.value] = ms
                idx += 3
            else:
                print("unknown parameters: ", scripts)
                raise ValueError(scripts[idx])
        self.result = {
            "ScriptsType": scriptsType.netem,
            "parameters": parameters
        }
        return

    def parse_tc(self):
        scripts = self.scripts
        if not scripts.startswith("tc qdisc add dev eth0 root "):
            return False
        scripts = scripts.removeprefix("tc qdisc add dev eth0 root ")
        if scripts.startswith('netem'):
            self.parse_tc_netem(scripts)
        elif scripts.startswith('tbf'):
            self.parser_tc_tbf(scripts)
        else:
            return False
        return True
    
    def parse_oss(self):
        scripts = self.scripts
        if not "download oss object" in scripts:
            return False
        parameters = [0 for i in range(len(ossOffset))]
        parameters[ossOffset.oss.value] = 1
        self.result = {
            "ScriptsType": scriptsType.oss,
            "parameters": parameters
        }
        return True
    
    def parse_hdd(self):
        # stress-ng --hdd 5 --hdd-bytes 338M --hdd-write-size 16k --timeout 204s
        scripts = self.scripts
        if not scripts.startswith('stress-ng --hdd '):
            return False
        scripts = scripts.removeprefix('stress-ng --hdd ').split(' ')
        parameters = [0 for i in range(len(hddOffset))]
        hdd = float(scripts[0])
        bytes = scripts[2]
        if bytes.endswith('M'):
            bytes = float(bytes[:-1])*1e3
        elif bytes.endswith('k'):
            bytes = float(bytes[:-1])
        else:
            raise ValueError("unknow hdd parameters:", bytes)
        size = scripts[4]
        if size.endswith('M'):
            size = float(size[:-1])*1e3
        elif size.endswith('k'):
            size = float(size[:-1])
        else:
            raise ValueError("unknow hdd parameters:", size)
        timeout = scripts[6]
        assert timeout.endswith('s')
        timeout = float(timeout[:-1])
        parameters[hddOffset.hdd.value] = hdd
        parameters[hddOffset.bytes.value] = bytes
        parameters[hddOffset.size.value] = size
        parameters[hddOffset.timeout.value] = timeout
        self.result = {
            "ScriptsType": scriptsType.hdd,
            "parameters": parameters
        }
        return True
    
    def parse_cpu(self):
        # stress-ng --cpu 8 --cpu-method floatconversion --timeout 106s
        scripts = self.scripts
        if not scripts.startswith('stress-ng --cpu '):
            return False
        scripts = scripts.removeprefix('stress-ng --cpu ').split(' ')
        parameters = [0 for i in range(len(cpuOffset))]
        cpu = float(scripts[0])
        parameters[cpuOffset.cpu.value] = cpu
        method = scripts[2]
        parameters[cpuOffset[method].value] = 1
        timeout = scripts[4]
        assert timeout.endswith('s')
        timeout = float(timeout[:-1])
        parameters[cpuOffset.timeout.value] = timeout
        self.result = {
            "ScriptsType": scriptsType.cpu,
            "parameters": parameters
        }
        return True
    
    def parse_vm(self):
        # stress-ng --vm 7 --vm-bytes 461M --vm-method move-inv --timeout 158s
        scripts = self.scripts
        if not scripts.startswith('stress-ng --vm '):
            return False
        scripts = scripts.removeprefix('stress-ng --vm ').split(' ')
        parameters = [0 for i in range(len(vmOffset))]
        vm = float(scripts[0])
        bytes = scripts[2]
        if bytes.endswith('M'):
            bytes = float(bytes[:-1])*1e3
        elif bytes.endswith('k'):
            bytes = float(bytes[:-1])
        else:
            raise ValueError("unknow hdd parameters:", bytes)
        method = scripts[4]
        timeout = scripts[6]
        assert timeout.endswith('s')
        timeout = float(timeout[:-1])
        parameters[vmOffset.vm.value] = vm
        parameters[vmOffset.bytes.value] = bytes
        parameters[vmOffset[method].value] = 1
        parameters[vmOffset.timeout.value] = timeout
        self.result = {
            "ScriptsType": scriptsType.vm,
            "parameters": parameters
        }
        return True

    def parse_tmp(self):
        scripts = self.scripts
        parameters = [0 for i in range(len(tmpOffset))]
        is_tmp = False
        if "write /tmp/tmp_write" in scripts:
            parameters[tmpOffset.writew.value] = 1
            is_tmp = True
        if "write /tmp/tmp_read" in scripts:
            parameters[tmpOffset.writer.value] = 1 
            is_tmp = True
        if "remove /tmp/tmp_write" in scripts:
            parameters[tmpOffset.removew.value] = 1
            is_tmp = True
        if "remove /tmp/tmp_read" in scripts:
            parameters[tmpOffset.remover.value] = 1 
            is_tmp = True
        if "\"/tmp/tmp_write" in scripts:
            parameters[tmpOffset.tmpw.value] = 1
            is_tmp = True
        if "\"/tmp/tmp_read" in scripts:
            parameters[tmpOffset.tmpr.value] = 1
            is_tmp = True
        if not is_tmp:
            return False
        self.result = {
            "ScriptsType": scriptsType.tmp,
            "parameters": parameters
        }
        return True
        
    
parser = ScriptsParser()

In [14]:
# pattern1 = "tc"
# mask1 = [True if i.startswith(pattern1) else False for i in workload.scripts]
# pattern2 = "tc qdisc add dev eth0 root netem"
# mask2 = [True if i.startswith(pattern2) else False for i in workload.scripts]
# pattern3 = "tc qdisc add dev eth0 root tbf"
# mask3 = [True if i.startswith(pattern3) else False for i in workload.scripts]
# m = [i or j for i, j in zip(mask2, mask3)]
# m == mask1
# tc命令有5种模式，
# tc qdisc add dev eth0 root netem loss x%
# tc qdisc add dev eth0 root netem duplicate x%
# tc qdisc add dev eth0 root netem delay xms
# tc qdisc add dev eth0 root netem reorder x%
# tc qdisc add dev eth0 root tbf rate xmbit brust ykbit latency zms

# pattern1 = "/tmp/tmp_"
# mask1 = [True if pattern1 in i else False for i in workload.scripts]

# scripts = workload.scripts[mask1]
# check = ""
# r = []
# for i in scripts:
#     if not parser.parse(i):
#         check = i
#         break
#     else:
#         r.append(parser.parse(i))
    

In [17]:
train_b.ips

['172.24.96.146',
 '172.24.96.147',
 '172.24.96.148',
 '172.24.96.149',
 '172.24.96.150',
 '172.24.96.151',
 '172.24.96.152',
 '172.24.96.153']

In [19]:
mask = train_b.train_df.instance_ip == train_b.ips[1]
for i in train_b.train_df[mask]:
    break

idxs = []
for ip in train_b.ips:
    mask = train_b.train_df.instance_ip == ip
    train_df = train_b.train_df[mask]
    mask = test_b.df.instance_ip == ip
    test_df = test_b.df[mask]
    train_end_time = test_df['event_time'].min()
    mask = train_df.event_time < train_end_time
    max = train_df[mask].sort_values(by='event_time')['event_time'].diff().max()
    print(max)

# 训练集中相邻状态时间差最大值
# 0 days 00:00:05
# 0 days 00:00:31
# 0 days 00:00:17
# 0 days 22:59:23
# 0 days 00:00:07
# 0 days 00:00:04
# 0 days 00:00:07
# 0 days 00:00:55
    

0 days 00:00:05
0 days 00:00:31
0 days 00:00:17
0 days 22:59:23
0 days 00:00:07
0 days 00:00:04
0 days 00:00:07
0 days 00:00:55


In [ ]:
# 按ip分为8个数据集
# 以train_df和test_df的event_time时间线，将时间线上每个有状态/需要求状态的时间点的workload标注出来
# 按test的最小时间为界，分为训练集和测试集

# 数据集格式
# x : [[meta], [jobType], [scriptsType], [previousState]], y: [indicatorsWanted]


In [16]:
train_df[mask]['event_time']

300       2024-07-11 17:15:02
301       2024-07-11 17:15:03
302       2024-07-11 17:15:04
303       2024-07-11 17:15:05
304       2024-07-11 17:15:06
                  ...        
9610863   2024-07-09 21:15:02
9610888   2024-07-12 21:00:02
9610977   2024-07-09 12:18:55
9610978   2024-07-09 12:19:00
9610979   2024-07-09 12:19:09
Name: event_time, Length: 657781, dtype: datetime64[ns]

In [20]:
train_b.train_df.iloc[3018]

event_time     2024-07-11 17:05:20
instance_ip          172.24.96.147
CPU                            all
%user                         0.25
%nice                          0.0
                      ...         
tcp-tw                        17.0
active/s                       1.0
passive/s                      0.0
iseg/s                         5.0
oseg/s                         8.0
Name: 3018, Length: 65, dtype: object

In [12]:
test = Test_b("round_b_test_v20240715", None)
test.load()

2024-07-28 05:39:42.305122 start load test  cpu.csv
2024-07-28 05:39:42.607289 start load test  disk.csv
2024-07-28 05:39:43.234688 start load test  memory.csv
2024-07-28 05:39:43.933888 start load test  network.csv


In [13]:
test.df

,instance_ip,event_time,cpu,disk,memory,network
0,172.24.96.146,2024-07-14 17:00:00,cpu,disk,memory,network
1,172.24.96.146,2024-07-14 17:00:01,cpu,NaN,memory,network
2,172.24.96.146,2024-07-14 17:00:02,cpu,disk,memory,network
3,172.24.96.146,2024-07-14 17:00:03,cpu,disk,memory,network
4,172.24.96.146,2024-07-14 17:00:04,cpu,disk,memory,network
...,...,...,...,...,...,...
775263,172.24.96.151,2024-07-15 09:05:01,NaN,NaN,memory,network
775264,172.24.96.152,2024-07-15 14:10:01,NaN,NaN,memory,network
775265,172.24.96.148,2024-07-15 06:43:42,NaN,NaN,NaN,network
775266,172.24.96.151,2024-07-14 19:00:02,NaN,NaN,NaN,network


In [30]:
train_b.train_df[CSV_FIELD['cpu.csv']]

,%user,%nice,%system,%iowait,%idle,proc/s,cswch/s
0,0.13,0.0,0.0,0.0,99.87,4.0,2522.0
1,0.0,0.0,0.0,0.0,100.0,0.0,2473.0
2,0.13,0.0,0.0,0.0,99.87,0.0,2615.0
3,0.12,0.0,0.12,0.0,99.75,0.0,2628.0
4,0.0,0.0,0.0,0.0,100.0,1.0,2507.0
...,...,...,...,...,...,...,...
9611065,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9611066,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9611067,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9611068,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
A = ["172.24.96.146", "172.24.96.147", "172.24.96.148", "172.24.96.149"]
manual_features_cpu_A = {
    "processors": 4,
    "cores": 2,
    "siblings": 4
}
B = ["172.24.96.150", "172.24.96.151", "172.24.96.152", "172.24.96.153"]
manual_features_cpu_B = {
    "processors": 8,
    "cores": 4,
    "siblings": 8
}

In [91]:
# read meta from file
dir = "round_b_machine_meta"
meta = {}
for item in os.listdir(dir):
    meta[item] = {}
    with open(os.path.join(dir, item), "r") as f:
        lines = f.readlines()
    ismem = False
    for line in lines:
        line = line.strip('\n')
        if line == "~# cat /proc/meminfo":
            ismem = True
            continue
        if not ismem:
            continue
        line = line.replace(" ", "").replace("kB", "").replace("\t", "").split(':')
        if len(line) != 2:
            continue
        meta[item][line[0]] = int(line[1])


In [92]:
ip0 = "172.24.96.146.txt"
ip1 = "172.24.96.150.txt"
zeros = []
for name in meta[ip1]:
    if meta[ip1][name] == 0 and meta[ip0][name] == 0:
        zeros.append(name)
zeros
for ip in meta:
    for name in zeros:
        del meta[ip][name]

In [94]:
manual_features_mem_A = meta[ip0]
manual_features_mem_B = meta[ip1]

In [98]:
len(manual_features_mem_A), len(manual_features_mem_B),len(manual_features_cpu_A), len(manual_features_cpu_B)
# meta feature for mem is 31, for cpu is 3

(31, 31, 3, 3)

In [99]:
train_b.train_df.sort_values(by='event_time')

,event_time,instance_ip,CPU,%user,%nice,%system,%iowait,%steal,%idle,proc/s,...,totsck,tcpsck,udpsck,rawsck,ip-frag,tcp-tw,active/s,passive/s,iseg/s,oseg/s
3201018,2024-07-08 09:55:02,172.24.96.149,all,80.95,0.0,17.54,0.0,0.0,1.5,0.0,...,162.0,15.0,3.0,0.0,0.0,1691.0,29.0,0.0,26878.0,17492.0
3201019,2024-07-08 09:55:03,172.24.96.149,all,80.05,0.0,13.97,0.0,0.0,5.99,0.0,...,162.0,16.0,3.0,0.0,0.0,1712.0,24.0,0.0,23581.0,15191.0
3201020,2024-07-08 09:55:04,172.24.96.149,all,82.83,0.0,11.87,0.0,0.0,5.3,0.0,...,162.0,16.0,3.0,0.0,0.0,1667.0,21.0,0.0,24017.0,15653.0
3201021,2024-07-08 09:55:05,172.24.96.149,all,81.45,0.0,13.53,0.0,0.0,5.01,0.0,...,164.0,18.0,3.0,0.0,0.0,1694.0,28.0,0.0,22439.0,14612.0
3201022,2024-07-08 09:55:06,172.24.96.149,all,83.33,0.0,13.13,0.0,0.0,3.54,0.0,...,164.0,17.0,3.0,0.0,0.0,1657.0,25.0,0.0,23279.0,15042.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9241813,2024-07-14 08:59:59,172.24.96.148,all,0.25,0.0,0.0,0.0,0.0,99.75,0.0,...,153.0,8.0,3.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0
9242111,2024-07-14 08:59:59,172.24.96.151,all,0.0,0.0,0.0,0.0,0.0,100.0,0.0,...,152.0,8.0,3.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0
9241217,2024-07-14 08:59:59,172.24.96.153,all,0.13,0.0,0.0,0.0,0.0,99.87,0.0,...,153.0,8.0,3.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
9242707,2024-07-14 08:59:59,172.24.96.146,all,0.25,0.0,0.0,0.0,0.0,99.75,0.0,...,153.0,8.0,3.0,0.0,0.0,11.0,1.0,0.0,17.0,13.0


,event_time,instance_ip,CPU,%user,%nice,%system,%iowait,%steal,%idle,proc/s,...,totsck,tcpsck,udpsck,rawsck,ip-frag,tcp-tw,active/s,passive/s,iseg/s,oseg/s
0,2024-07-11 17:00:02,172.24.96.150,all,0.13,0.0,0.0,0.0,0.0,99.87,4.0,...,153.0,8.0,3.0,0.0,0.0,10.0,0.0,0.0,15.0,19.0
1,2024-07-11 17:00:03,172.24.96.150,all,0.0,0.0,0.0,0.0,0.0,100.0,0.0,...,153.0,8.0,3.0,0.0,0.0,10.0,0.0,0.0,3.0,5.0
2,2024-07-11 17:00:04,172.24.96.150,all,0.13,0.0,0.0,0.0,0.0,99.87,0.0,...,153.0,8.0,3.0,0.0,0.0,10.0,0.0,0.0,4.0,8.0
3,2024-07-11 17:00:05,172.24.96.150,all,0.12,0.0,0.12,0.0,0.0,99.75,0.0,...,153.0,8.0,3.0,0.0,0.0,10.0,0.0,0.0,4.0,8.0
4,2024-07-11 17:00:06,172.24.96.150,all,0.0,0.0,0.0,0.0,0.0,100.0,1.0,...,153.0,8.0,3.0,0.0,0.0,13.0,3.0,0.0,24.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9611065,2024-07-09 11:43:59,172.24.96.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
9611066,2024-07-09 11:44:10,172.24.96.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
9611067,2024-07-09 11:44:21,172.24.96.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0
9611068,2024-07-09 11:44:48,172.24.96.147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.1,0.1
